In [0]:
# Importando os tipos de dados necessários
from pyspark.sql.functions import col, sum, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# 1. Definindo os dados dos produtos
dados_produtos = [
    (1, "Notebook Gamer", "Eletrônicos", 8500.00),
    (2, "Cadeira de Escritório", "Móveis", 1200.50),
    (3, "Monitor 4K", "Eletrônicos", 2500.00),
    (4, "Teclado Mecânico", "Acessórios", 350.75)
]
dados_vendas = [
    (101, 1, 1, "2025-09-20"),
    (102, 3, 2, "2025-09-21"),
    (103, 2, 1, "2025-09-21"),
    (104, 1, 1, "2025-09-22"),
    (105, 4, 5, "2025-09-23")
]

# 2. Definindo o Schema (a estrutura da tabela)
schema_produtos = StructType([
    StructField("id_produto", IntegerType(), True),
    StructField("nome_produto", StringType(), True),
    StructField("categoria", StringType(), True),
    StructField("preco", DoubleType(), True)
])
schema_vendas = StructType([
    StructField("id_venda", IntegerType(), True),
    StructField("id_produto", IntegerType(), True),
    StructField("quantidade", IntegerType(), True),
    StructField("data_venda", StringType(), True) 
])

# 3. Criando o DataFrame
df_produtos = spark.createDataFrame(data=dados_produtos, schema=schema_produtos)

# 4. Exibindo os dados (Ação)
print("Visualização dos dados dos produtos:")
df_produtos.show()

# 5. Exibindo a estrutura (Schema)
print("Schema do DataFrame de produtos:")
df_produtos.printSchema()

df_vendas = spark.createDataFrame(data=dados_vendas, schema=schema_vendas)
df_relatorio = df_vendas.join(df_produtos, on="id_produto", how="inner")
print("Dados após o JOIN:")
df_relatorio.show()
df_relatorio = df_relatorio.withColumn("valor_total_venda", col("quantidade")* col("preco"))
print("Dados com a nova coluna 'valor_total_venda':")
df_relatorio.show()

df_final = df_relatorio.groupBy("categoria") \
                       .agg(
                           sum("valor_total_venda").alias("total_vendido"),
                           count("id_venda").alias("numero_de_vendas")
                       )

# 5. Mostrando o resultado
print("Relatório Final de Vendas por Categoria:")
df_final.show()
